In [81]:
import pandas as pd
from pytrends.request import TrendReq
import time
import json

# Load the JSON file
with open('output.json', 'r') as file:
    uptownfunk_data = json.load(file)

# Extract unique words from the JSON content
keywords = list(set(word.lower() for entry in uptownfunk_data if 'text' in entry for word in entry['text'].split()))
regions = [
    "IN" 
]
timeframe = "now 7-d"  

In [82]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
custom_stop_words = {'ago', 'am', 'i', 'you', 'me', 'your', 'it', 'for', 'to', 'of', 'in', 'on', 'with', 
                     'the', 'this', 'that', 'my', 'we', 'who', 'if', 'about', 'up', 'out', 'got', 
                     'get', 'what', 'a', 'also', 'from', 'and', 'or', 'but', 'so', 'just', 'at',
                     'be', 'is', 'are', 'was', 'were', 'been', 'do', 'does', 'did', 'will', 'would',
                     'should', 'can', 'could', 'may', 'might', 'must', 'shall', 'having', 'has', 'had',
                     'how', 'why', 'when', 'where', 'which', 'whom', 'whose', 'here', 'there', 'then',
                     'than', 'now', 'over', 'under', 'between', 'before', 'after', 'while', 'during',
                     'against', 'through', 'though', 'since', 'because', 'until', 'unless', 'nor', 'not',
                     'no', 'yes', 'yeah', 'yep', 'yup', 'ok', 'okay', 'sure', 'well', 'good', 'great',
                     'nice', 'awesome', 'cool', 'bad', 'terrible', 'horrible', 'awful', 'wonderful', 'fantastic',}
stop_words.update(custom_stop_words)

def clean_keyword(keyword):
    keyword = re.sub(r'[^\w\s]', '', keyword)
    keyword = keyword.lower()
    if keyword in stop_words or len(keyword) < 2:
        return None
    return keyword

keywords = {clean_keyword(kw) for kw in keywords if clean_keyword(kw)}

In [83]:
keywords

{'problem',
 'poison',
 'opportunity',
 'dream',
 'run',
 'pair',
 'milkshake',
 'around',
 'publish',
 'easy',
 'couple',
 'based',
 'told',
 'extends',
 'bell',
 'smart',
 'needing',
 'virtual',
 'directly',
 'reach',
 'write',
 'store',
 'work',
 'summer',
 'gra',
 'think',
 'explanation',
 'acting',
 'backwards',
 'news',
 'plausible',
 'pedometer',
 'bacon',
 'different',
 'butterfly',
 'world',
 'saturday',
 'half',
 'wash',
 'realize',
 'freshener',
 'life',
 'silly',
 'otherwise',
 'nothings',
 'smell',
 'give',
 'mine',
 'birth',
 'michael',
 'frustrated',
 'business',
 'crammed',
 'staff',
 'beautiful',
 'enough',
 'ideas',
 'obsessed',
 'whole',
 'encounters',
 'still',
 'theres',
 'lactose',
 'ive',
 'care',
 'theoretical',
 'bed',
 'breakfast',
 'awkward',
 'marketable',
 'bedroom',
 'open',
 'fire',
 'thing',
 'pocket',
 'raj',
 'made',
 'lucky',
 'implying',
 'master',
 'parallax',
 'security',
 'go',
 'invite',
 'pacino',
 'conclusions',
 'bickering',
 'los',
 'currentl

In [84]:
def chunk_keywords(keywords, chunk_size):
    """
    Splits a list of keywords into chunks of a specified size.
    
    Args:
        keywords (list): List of keywords to split.
        chunk_size (int): Size of each chunk.
    
    Returns:
        list: List of keyword chunks.
    """
    for i in range(0, len(keywords), chunk_size):
        yield keywords[i:i + chunk_size]

In [85]:
def analyze_search_trends(keywords, regions, timeframe):
    """
    Analyzes search trends for given keywords and regions with delays to avoid rate limits.
    
    Args:
        keywords (list): List of keywords to analyze (up to 5 per request).
        regions (list): List of region codes (e.g., 'US', 'UK').
        timeframe (str): Timeframe for trends (e.g., 'today 5-y').
    
    Returns:
        dict: Trend data for each region.
    """
    pytrends = TrendReq(hl='en-US', tz=360)  # Initialize pytrends
    all_data = {}

    # Split keywords into chunks of 5
    keyword_chunks = chunk_keywords(keywords, chunk_size=5)

    flag = 0

    for region in regions:
        for chunk in keyword_chunks:
            try:
                flag += 1
                # Build payload and fetch data
                pytrends.build_payload(chunk, cat=0, timeframe=timeframe, geo=region)
                data = pytrends.interest_over_time()
                if not data.empty:
                    if region not in all_data:
                        all_data[region] = data
                    else:
                        all_data[region] = all_data[region].join(data, how='outer', rsuffix='_dup')
                
                # Pause for 5 seconds to respect rate limits
                time.sleep(5)
            
            except Exception as e:
                flag += 1

                print(f"Error fetching data for {region} with keywords {chunk}: {e}")
                # Wait longer if an error occurs, then continue
                time.sleep(10)

            finally:
                if flag == 3:
                    break
    
    return all_data


In [86]:
# Convert keywords set to a list for chunking
keywords_list = list(keywords)

# Execute analysis
trend_data = analyze_search_trends(keywords_list, regions, timeframe)


# Display results
for region, data in trend_data.items():
    print(f"Trends for {region}:")
    print(data.head())
    print("\n")


C:\Users\Akshat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Akshat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Akshat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\Loca

Trends for IN:
                     problem  poison  opportunity  dream  run  isPartial  \
date                                                                       
2025-03-14 11:00:00       70       7            6     45   69      False   
2025-03-14 12:00:00       74       7            6     48   74      False   
2025-03-14 13:00:00       76       7            6     48   72      False   
2025-03-14 14:00:00       78       7            7     44   78      False   
2025-03-14 15:00:00       77       7            6     44   62      False   

                     pair  milkshake  around  publish  easy  isPartial_dup  \
date                                                                         
2025-03-14 11:00:00    17          2      20        2    65          False   
2025-03-14 12:00:00    18          2      20        2    73          False   
2025-03-14 13:00:00    19          2      21        2    75          False   
2025-03-14 14:00:00    20          2      22        2    87   

In [87]:
data

,problem,poison,opportunity,dream,run,isPartial,pair,milkshake,around,publish,easy,isPartial_dup,couple,based,told,extends,bell,isPartial_dup
date,,,,,,,,,,,,,,,,,,
2025-03-14 11:00:00,70,7,6,45,69,False,17,2,20,2,65,False,36,30,2,0,7,False
2025-03-14 12:00:00,74,7,6,48,74,False,18,2,20,2,73,False,32,30,3,0,8,False
2025-03-14 13:00:00,76,7,6,48,72,False,19,2,21,2,75,False,26,31,3,0,7,False
2025-03-14 14:00:00,78,7,7,44,78,False,20,2,22,2,87,False,28,27,3,0,8,False
2025-03-14 15:00:00,77,7,6,44,62,False,19,2,20,2,73,False,28,27,3,0,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21 07:00:00,67,5,9,49,67,False,14,1,17,2,54,False,24,32,2,0,6,False
2025-03-21 08:00:00,71,5,9,50,62,False,18,1,15,2,58,False,28,30,2,0,6,False
2025-03-21 09:00:00,67,4,8,49,67,False,15,2,15,2,60,False,27,30,2,0,6,False


In [ ]:
The scores in the trends data represent the relative search interest for the given keywords over time, as provided by Google Trends. Here's what they mean:

1. **Score Values**: Each score is a normalized value between 0 and 100, where:
    - `100` represents the peak popularity of the keyword during the specified timeframe.
    - `0` means there was insufficient data for that keyword at that time.

2. **Columns**: Each column corresponds to a keyword (e.g., `problem`, `poison`, etc.), and the values indicate the search interest for that keyword at the given timestamp.

3. **`isPartial` Columns**: These boolean columns indicate whether the data for that time period is incomplete or partial.

In summary, these scores help you understand how popular each keyword was in the specified region (`IN`) and timeframe (`now 7-d`).